In [1]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding
import numpy as np

batch_size = 64  # Batch size for training.
epochs = 20  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 65

Using TensorFlow backend.


In [2]:
input_texts = []

target_texts = []

input_words = set()

target_words = set()

In [3]:
with open("one.tsv", 'r', encoding='utf-8') as f:

    lines = f.readlines()

In [4]:
#for row in range(df.shape(0)):
for line in lines[: min(num_samples, len(lines) - 1)]:

    
    tst = line.split('\t')
    input_text = tst[0]
    target_text_0 = tst[1].strip()
    target_text= target_text_0.strip()
    
    # We use "tab" as the "start sequence" character

    # for the targets, and "\n" as "end sequence" character.
    #input_text = df.iloc[row,0]
    target_text = target_text 

    input_texts.append(input_text)

    target_texts.append(target_text)

    for word in input_text.split(" "):

        if word not in input_words:

            input_words.add(word)

    for word in target_text.split(" "):

        if word not in target_words:

            target_words.add(word)

In [5]:
target_words.add("START_")
target_words.add("_END")

In [6]:
input_words = sorted(list(input_words))

target_words = sorted(list(target_words))

num_encoder_tokens = len(input_words)

num_decoder_tokens = len(target_words)

max_encoder_seq_length = max([len(txt.split(' ')) for txt in input_texts])

max_decoder_seq_length = max([len(txt.split(' ')) for txt in target_texts])

In [7]:
print('Number of samples:', len(input_texts))

print('Number of unique input tokens:', num_encoder_tokens)

print('Number of unique output tokens:', num_decoder_tokens)

print('Max sequence length for inputs:', max_encoder_seq_length)

print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 65
Number of unique input tokens: 72
Number of unique output tokens: 66
Max sequence length for inputs: 7
Max sequence length for outputs: 7


In [8]:
input_token_index = dict(

    [(word, i) for i, word in enumerate(input_words)])

target_token_index = dict(

    [(word, i) for i, word in enumerate(target_words)])

In [9]:
target_token_index

{'Agra': 0,
 'Amaravathi': 1,
 'Bengaluru': 2,
 'Chennai': 3,
 'Fine': 4,
 'Gandinagar': 5,
 'Hello': 6,
 'Hi': 7,
 'Hot': 8,
 'Humid': 9,
 'Hyderabad': 10,
 'I': 11,
 'Kolkatta': 12,
 'Laknow': 13,
 'Mumbai': 14,
 'START_': 15,
 'Very': 16,
 '_END': 17,
 'afternoon': 18,
 'am': 19,
 'amsterdam': 20,
 'are': 21,
 'bengali': 22,
 'berlin': 23,
 'buvaneshwar': 24,
 'bye': 25,
 'chandigarh': 26,
 'cochin': 27,
 'colombo': 28,
 'dispur': 29,
 'doing?': 30,
 'evening': 31,
 'fine,': 32,
 'good': 33,
 'good,': 34,
 'gujarati': 35,
 'hello': 36,
 'helsinki': 37,
 'hi': 38,
 'hindi': 39,
 'hot': 40,
 'how': 41,
 'hyderabad': 42,
 'jaipur': 43,
 'jammu': 44,
 'kannada': 45,
 'kaulalumpur': 46,
 'london': 47,
 'malayalam': 48,
 'marati': 49,
 'morning': 50,
 'nagaland': 51,
 'oriya': 52,
 'ottawa': 53,
 'output': 54,
 'paris': 55,
 'punjabi': 56,
 'rome': 57,
 'spicy': 58,
 'tamil': 59,
 'telugu': 60,
 'there': 61,
 'washington': 62,
 'you': 63,
 'you?': 64,
 'yummy': 65}

In [10]:
import numpy as np

In [11]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(target_texts),  max_decoder_seq_length*2),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(target_texts), max_decoder_seq_length*2, num_decoder_tokens),
    dtype='float32')
"""
encoder_input_data = np.zeros( (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')

decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')

decoder_target_data = np.zeros( (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')
"""

"\nencoder_input_data = np.zeros( (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32')\n\ndecoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')\n\ndecoder_target_data = np.zeros( (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32')\n"

In [12]:
input_texts_1=input_texts[1:]
target_texts_0=target_texts[1:]


In [13]:
target_texts_1 = []
for i in range(len(target_texts_0)):
    target_texts_1.append(str('START_ ')+ str(target_texts_0[i])+ str(' _END'))

In [14]:
input_texts_1

['Hello',
 'Hi',
 'How are you?',
 'Hi there',
 'How are you doing?',
 'How is the summer weather in Hyderabad?',
 'How is the summer weather in delhi?',
 'How is the weather in chennai?',
 'where is Tajmahal?',
 'How is the icecream?',
 'how is the biryani?',
 'Where is charminar?',
 'which language in telangana?',
 'which language in andhra?',
 'which language in tamilnadu?',
 'which language in karnataka?',
 'which language in kerala?',
 'which language in maharashtra?',
 'which language in orissa?',
 'which language in bengal?',
 'which language in punjab?',
 'which language in uttar pradesh?',
 'which language in madhya pradesh?',
 'which language in himachal pradesh?',
 'which language in haryana?',
 'which language in uttarakand?',
 'which language in gujarat?',
 'Capital of Telangana?',
 'Capital of Andhra pradesh?',
 'Capital of Karnataka?',
 'Capital of Tamilnadu?',
 'Capital of kerala?',
 'capital of maharashtra?',
 'Capital of Uttar pradesh?',
 'Capital of bengal?',
 'Capit

In [15]:
target_texts_1

['START_ Hello _END',
 'START_ Hi _END',
 'START_ Fine _END',
 'START_ Hi there _END',
 'START_ Fine _END',
 'START_ Hot _END',
 'START_ Very hot _END',
 'START_ Humid _END',
 'START_ Agra _END',
 'START_ yummy _END',
 'START_ spicy _END',
 'START_ hyderabad _END',
 'START_ telugu _END',
 'START_ telugu _END',
 'START_ tamil _END',
 'START_ kannada _END',
 'START_ malayalam _END',
 'START_ marati _END',
 'START_ oriya _END',
 'START_ bengali _END',
 'START_ punjabi _END',
 'START_ hindi _END',
 'START_ hindi _END',
 'START_ hindi _END',
 'START_ hindi _END',
 'START_ hindi _END',
 'START_ gujarati _END',
 'START_ Hyderabad _END',
 'START_ Amaravathi _END',
 'START_ Bengaluru _END',
 'START_ Chennai _END',
 'START_ cochin _END',
 'START_ Mumbai _END',
 'START_ Laknow _END',
 'START_ Kolkatta _END',
 'START_ chandigarh _END',
 'START_ Gandinagar _END',
 'START_ chandigarh _END',
 'START_ buvaneshwar _END',
 'START_ jammu _END',
 'START_ jaipur _END',
 'START_ nagaland _END',
 'START_ dis

In [16]:
for i, (input_text, target_text) in enumerate(zip(input_texts_1, target_texts_1)):
    for t, word in enumerate(input_text.split()):
        encoder_input_data[i, t] = input_token_index[word]
    for t, word in enumerate(target_text.split()):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        print
        decoder_input_data[i, t] = target_token_index[word]
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[word]] = 1.


In [17]:
target_token_index

{'Agra': 0,
 'Amaravathi': 1,
 'Bengaluru': 2,
 'Chennai': 3,
 'Fine': 4,
 'Gandinagar': 5,
 'Hello': 6,
 'Hi': 7,
 'Hot': 8,
 'Humid': 9,
 'Hyderabad': 10,
 'I': 11,
 'Kolkatta': 12,
 'Laknow': 13,
 'Mumbai': 14,
 'START_': 15,
 'Very': 16,
 '_END': 17,
 'afternoon': 18,
 'am': 19,
 'amsterdam': 20,
 'are': 21,
 'bengali': 22,
 'berlin': 23,
 'buvaneshwar': 24,
 'bye': 25,
 'chandigarh': 26,
 'cochin': 27,
 'colombo': 28,
 'dispur': 29,
 'doing?': 30,
 'evening': 31,
 'fine,': 32,
 'good': 33,
 'good,': 34,
 'gujarati': 35,
 'hello': 36,
 'helsinki': 37,
 'hi': 38,
 'hindi': 39,
 'hot': 40,
 'how': 41,
 'hyderabad': 42,
 'jaipur': 43,
 'jammu': 44,
 'kannada': 45,
 'kaulalumpur': 46,
 'london': 47,
 'malayalam': 48,
 'marati': 49,
 'morning': 50,
 'nagaland': 51,
 'oriya': 52,
 'ottawa': 53,
 'output': 54,
 'paris': 55,
 'punjabi': 56,
 'rome': 57,
 'spicy': 58,
 'tamil': 59,
 'telugu': 60,
 'there': 61,
 'washington': 62,
 'you': 63,
 'you?': 64,
 'yummy': 65}

In [18]:
embedding_size=latent_dim
encoder_inputs = Input(shape=(None,))
en_x=  Embedding(num_encoder_tokens, embedding_size)(encoder_inputs)
encoder = LSTM(50, return_state=True)
encoder_outputs, state_h, state_c = encoder(en_x)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]




# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))

dex=  Embedding(num_decoder_tokens, embedding_size)

final_dex= dex(decoder_inputs)


decoder_lstm = LSTM(50, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(final_dex,
                                     initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')

decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['acc'])

"""
encoder_inputs = Input(shape=(None,))
x = Embedding(num_encoder_tokens, latent_dim)(encoder_inputs)
x, state_h, state_c = LSTM(latent_dim,
                           return_state=True)(x)
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
x = Embedding(num_decoder_tokens, latent_dim)(decoder_inputs)
x = LSTM(latent_dim, return_sequences=True)(x, initial_state=encoder_states)
decoder_outputs = Dense(num_decoder_tokens, activation='softmax')(x)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Compile & run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')
"""

"\nencoder_inputs = Input(shape=(None,))\nx = Embedding(num_encoder_tokens, latent_dim)(encoder_inputs)\nx, state_h, state_c = LSTM(latent_dim,\n                           return_state=True)(x)\nencoder_states = [state_h, state_c]\n\n# Set up the decoder, using `encoder_states` as initial state.\ndecoder_inputs = Input(shape=(None,))\nx = Embedding(num_decoder_tokens, latent_dim)(decoder_inputs)\nx = LSTM(latent_dim, return_sequences=True)(x, initial_state=encoder_states)\ndecoder_outputs = Dense(num_decoder_tokens, activation='softmax')(x)\n\n# Define the model that will turn\n# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`\nmodel = Model([encoder_inputs, decoder_inputs], decoder_outputs)\n\n# Compile & run training\nmodel.compile(optimizer='rmsprop', loss='categorical_crossentropy')\n"

In [19]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=500,
          validation_split=0.2)

Train on 52 samples, validate on 13 samples
Epoch 1/500
52/52 [==============================] - 2s 39ms/step - loss: 0.6160 - acc: 0.0014 - val_loss: 0.8912 - val_acc: 0.0659
Epoch 2/500
52/52 [==============================] - 0s 2ms/step - loss: 0.6077 - acc: 0.0714 - val_loss: 0.8853 - val_acc: 0.0659
Epoch 3/500
52/52 [==============================] - 0s 1ms/step - loss: 0.5998 - acc: 0.0714 - val_loss: 0.8775 - val_acc: 0.0659
Epoch 4/500
52/52 [==============================] - 0s 2ms/step - loss: 0.5896 - acc: 0.0714 - val_loss: 0.8666 - val_acc: 0.0659
Epoch 5/500
52/52 [==============================] - 0s 1ms/step - loss: 0.5756 - acc: 0.0714 - val_loss: 0.8512 - val_acc: 0.0659
Epoch 6/500
52/52 [==============================] - 0s 1ms/step - loss: 0.5562 - acc: 0.0714 - val_loss: 0.8298 - val_acc: 0.0659
Epoch 7/500
52/52 [==============================] - 0s 2ms/step - loss: 0.5294 - acc: 0.0714 - val_loss: 0.8027 - val_acc: 0.0659
Epoch 8/500
52/52 [===================

Epoch 63/500
52/52 [==============================] - 0s 1ms/step - loss: 0.2576 - acc: 0.0755 - val_loss: 1.0231 - val_acc: 0.0659
Epoch 64/500
52/52 [==============================] - 0s 2ms/step - loss: 0.2548 - acc: 0.0728 - val_loss: 0.9980 - val_acc: 0.0659
Epoch 65/500
52/52 [==============================] - 0s 2ms/step - loss: 0.2521 - acc: 0.0838 - val_loss: 1.0377 - val_acc: 0.0659
Epoch 66/500
52/52 [==============================] - 0s 1ms/step - loss: 0.2498 - acc: 0.0728 - val_loss: 0.9980 - val_acc: 0.0659
Epoch 67/500
52/52 [==============================] - 0s 1ms/step - loss: 0.2473 - acc: 0.0852 - val_loss: 1.0493 - val_acc: 0.0659
Epoch 68/500
52/52 [==============================] - 0s 1ms/step - loss: 0.2448 - acc: 0.0728 - val_loss: 1.0055 - val_acc: 0.0659
Epoch 69/500
52/52 [==============================] - 0s 1ms/step - loss: 0.2414 - acc: 0.0879 - val_loss: 1.0473 - val_acc: 0.0659
Epoch 70/500
52/52 [==============================] - 0s 2ms/step - loss: 0.

Epoch 125/500
52/52 [==============================] - 0s 2ms/step - loss: 0.0861 - acc: 0.1429 - val_loss: 1.0640 - val_acc: 0.0549
Epoch 126/500
52/52 [==============================] - 0s 1ms/step - loss: 0.0841 - acc: 0.1442 - val_loss: 1.0923 - val_acc: 0.0714
Epoch 127/500
52/52 [==============================] - 0s 1ms/step - loss: 0.0820 - acc: 0.1442 - val_loss: 1.0710 - val_acc: 0.0549
Epoch 128/500
52/52 [==============================] - 0s 2ms/step - loss: 0.0800 - acc: 0.1442 - val_loss: 1.0877 - val_acc: 0.0714
Epoch 129/500
52/52 [==============================] - 0s 2ms/step - loss: 0.0783 - acc: 0.1442 - val_loss: 1.0794 - val_acc: 0.0549
Epoch 130/500
52/52 [==============================] - 0s 1ms/step - loss: 0.0766 - acc: 0.1429 - val_loss: 1.0882 - val_acc: 0.0714
Epoch 131/500
52/52 [==============================] - 0s 2ms/step - loss: 0.0748 - acc: 0.1442 - val_loss: 1.0848 - val_acc: 0.0549
Epoch 132/500
52/52 [==============================] - 0s 1ms/step - 

Epoch 187/500
52/52 [==============================] - 0s 2ms/step - loss: 0.0159 - acc: 0.1470 - val_loss: 1.2154 - val_acc: 0.0440
Epoch 188/500
52/52 [==============================] - 0s 2ms/step - loss: 0.0154 - acc: 0.1470 - val_loss: 1.1982 - val_acc: 0.0440
Epoch 189/500
52/52 [==============================] - 0s 2ms/step - loss: 0.0150 - acc: 0.1470 - val_loss: 1.2195 - val_acc: 0.0440
Epoch 190/500
52/52 [==============================] - 0s 1ms/step - loss: 0.0146 - acc: 0.1470 - val_loss: 1.1983 - val_acc: 0.0440
Epoch 191/500
52/52 [==============================] - 0s 2ms/step - loss: 0.0142 - acc: 0.1470 - val_loss: 1.2211 - val_acc: 0.0440
Epoch 192/500
52/52 [==============================] - 0s 1ms/step - loss: 0.0138 - acc: 0.1470 - val_loss: 1.1982 - val_acc: 0.0440
Epoch 193/500
52/52 [==============================] - 0s 1ms/step - loss: 0.0135 - acc: 0.1470 - val_loss: 1.2210 - val_acc: 0.0440
Epoch 194/500
52/52 [==============================] - 0s 1ms/step - 

Epoch 249/500
52/52 [==============================] - 0s 2ms/step - loss: 0.0027 - acc: 0.1470 - val_loss: 1.3434 - val_acc: 0.0440
Epoch 250/500
52/52 [==============================] - 0s 2ms/step - loss: 0.0026 - acc: 0.1470 - val_loss: 1.3279 - val_acc: 0.0440
Epoch 251/500
52/52 [==============================] - 0s 2ms/step - loss: 0.0025 - acc: 0.1470 - val_loss: 1.3444 - val_acc: 0.0440
Epoch 252/500
52/52 [==============================] - 0s 2ms/step - loss: 0.0024 - acc: 0.1470 - val_loss: 1.3408 - val_acc: 0.0440
Epoch 253/500
52/52 [==============================] - 0s 2ms/step - loss: 0.0024 - acc: 0.1470 - val_loss: 1.3473 - val_acc: 0.0440
Epoch 254/500
52/52 [==============================] - 0s 2ms/step - loss: 0.0023 - acc: 0.1470 - val_loss: 1.3523 - val_acc: 0.0440
Epoch 255/500
52/52 [==============================] - 0s 1ms/step - loss: 0.0022 - acc: 0.1470 - val_loss: 1.3531 - val_acc: 0.0440
Epoch 256/500
52/52 [==============================] - 0s 1ms/step - 

Epoch 310/500
52/52 [==============================] - 0s 2ms/step - loss: 5.2092e-04 - acc: 0.1470 - val_loss: 1.5271 - val_acc: 0.0440
Epoch 311/500
52/52 [==============================] - 0s 1ms/step - loss: 5.0210e-04 - acc: 0.1470 - val_loss: 1.5499 - val_acc: 0.0440
Epoch 312/500
52/52 [==============================] - 0s 2ms/step - loss: 4.8876e-04 - acc: 0.1470 - val_loss: 1.5418 - val_acc: 0.0440
Epoch 313/500
52/52 [==============================] - 0s 2ms/step - loss: 4.7401e-04 - acc: 0.1470 - val_loss: 1.5538 - val_acc: 0.0440
Epoch 314/500
52/52 [==============================] - 0s 1ms/step - loss: 4.6113e-04 - acc: 0.1470 - val_loss: 1.5493 - val_acc: 0.0440
Epoch 315/500
52/52 [==============================] - 0s 2ms/step - loss: 4.4909e-04 - acc: 0.1470 - val_loss: 1.5656 - val_acc: 0.0440
Epoch 316/500
52/52 [==============================] - 0s 2ms/step - loss: 4.3795e-04 - acc: 0.1470 - val_loss: 1.5551 - val_acc: 0.0440
Epoch 317/500
52/52 [====================

Epoch 370/500
52/52 [==============================] - 0s 1ms/step - loss: 1.0818e-04 - acc: 0.1470 - val_loss: 1.7390 - val_acc: 0.0440
Epoch 371/500
52/52 [==============================] - 0s 2ms/step - loss: 1.0513e-04 - acc: 0.1470 - val_loss: 1.7352 - val_acc: 0.0440
Epoch 372/500
52/52 [==============================] - 0s 1ms/step - loss: 1.0242e-04 - acc: 0.1470 - val_loss: 1.7513 - val_acc: 0.0440
Epoch 373/500
52/52 [==============================] - 0s 1ms/step - loss: 9.9892e-05 - acc: 0.1470 - val_loss: 1.7434 - val_acc: 0.0440
Epoch 374/500
52/52 [==============================] - 0s 2ms/step - loss: 9.7442e-05 - acc: 0.1470 - val_loss: 1.7638 - val_acc: 0.0440
Epoch 375/500
52/52 [==============================] - 0s 2ms/step - loss: 9.5218e-05 - acc: 0.1470 - val_loss: 1.7521 - val_acc: 0.0440
Epoch 376/500
52/52 [==============================] - 0s 2ms/step - loss: 9.2591e-05 - acc: 0.1470 - val_loss: 1.7690 - val_acc: 0.0440
Epoch 377/500
52/52 [====================

Epoch 430/500
52/52 [==============================] - 0s 3ms/step - loss: 2.5260e-05 - acc: 0.1470 - val_loss: 1.8657 - val_acc: 0.0440
Epoch 431/500
52/52 [==============================] - 0s 2ms/step - loss: 2.3646e-05 - acc: 0.1470 - val_loss: 1.9207 - val_acc: 0.0440
Epoch 432/500
52/52 [==============================] - 0s 2ms/step - loss: 2.2836e-05 - acc: 0.1470 - val_loss: 1.8897 - val_acc: 0.0440
Epoch 433/500
52/52 [==============================] - 0s 3ms/step - loss: 2.2123e-05 - acc: 0.1470 - val_loss: 1.9119 - val_acc: 0.0440
Epoch 434/500
52/52 [==============================] - 0s 2ms/step - loss: 2.1558e-05 - acc: 0.1470 - val_loss: 1.9061 - val_acc: 0.0440
Epoch 435/500
52/52 [==============================] - 0s 3ms/step - loss: 2.1045e-05 - acc: 0.1470 - val_loss: 1.9192 - val_acc: 0.0440
Epoch 436/500
52/52 [==============================] - 0s 2ms/step - loss: 2.0547e-05 - acc: 0.1470 - val_loss: 1.9109 - val_acc: 0.0440
Epoch 437/500
52/52 [====================

Epoch 490/500
52/52 [==============================] - 0s 1ms/step - loss: 5.6647e-06 - acc: 0.1470 - val_loss: 2.0087 - val_acc: 0.0440
Epoch 491/500
52/52 [==============================] - 0s 1ms/step - loss: 5.5145e-06 - acc: 0.1470 - val_loss: 2.0098 - val_acc: 0.0440
Epoch 492/500
52/52 [==============================] - 0s 1ms/step - loss: 5.3778e-06 - acc: 0.1470 - val_loss: 2.0106 - val_acc: 0.0440
Epoch 493/500
52/52 [==============================] - 0s 1ms/step - loss: 5.2492e-06 - acc: 0.1470 - val_loss: 2.0171 - val_acc: 0.0440
Epoch 494/500
52/52 [==============================] - 0s 1ms/step - loss: 5.1338e-06 - acc: 0.1470 - val_loss: 2.0120 - val_acc: 0.0440
Epoch 495/500
52/52 [==============================] - 0s 1ms/step - loss: 5.0176e-06 - acc: 0.1470 - val_loss: 2.0185 - val_acc: 0.0440
Epoch 496/500
52/52 [==============================] - 0s 2ms/step - loss: 4.9060e-06 - acc: 0.1470 - val_loss: 2.0127 - val_acc: 0.0440
Epoch 497/500
52/52 [====================

In [20]:
model.save('s2s_65.h5')

C:\Users\GGS\Anaconda3\envs\tfdeeplearning_1\lib\site-packages\keras\engine\topology.py:2344: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 50) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 50) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  str(node.arguments) + '. They will not be included '


In [21]:
# define the encoder model 
encoder_model = Model(encoder_inputs, encoder_states)
encoder_model.summary()
# Redefine the decoder model with decoder will be getting below inputs from encoder while in prediction
decoder_state_input_h = Input(shape=(50,))
decoder_state_input_c = Input(shape=(50,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
final_dex2= dex(decoder_inputs)
decoder_outputs2, state_h2, state_c2 = decoder_lstm(final_dex2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2)
# sampling model will take encoder states and decoder_input(seed initially) and output the predictions(french word index) We dont care about decoder_states2
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, None)              0         
_________________________________________________________________
embedding_1 (Embedding)      (None, None, 256)         18432     
_________________________________________________________________
lstm_1 (LSTM)                [(None, 50), (None, 50),  61400     
Total params: 79,832
Trainable params: 79,832
Non-trainable params: 0
_________________________________________________________________


In [22]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 52):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [23]:
for seq_index in range(30,50,1):

    # Take one sequence (part of the training set)

    # for trying out decoding.

    input_seq = encoder_input_data[seq_index: seq_index + 1]
    #input_seq = encoder_input_data[[seq_index]]
    #print(input_seq)
    #input_sample = input_texts[seq_index]
    #input_seq = [encoder_input_data for range]

    decoded_sentence = decode_sequence(input_seq)

    print('-')

    print('Input sentence:', input_texts_1[seq_index])

    print('Decoded sentence:', decoded_sentence.strip("_END"))

-
Input sentence: Capital of Tamilnadu?
Decoded sentence:  Chennai 
-
Input sentence: Capital of kerala?
Decoded sentence:  cochin 
-
Input sentence: capital of maharashtra?
Decoded sentence:  Mumbai 
-
Input sentence: Capital of Uttar pradesh?
Decoded sentence:  Laknow 
-
Input sentence: Capital of bengal?
Decoded sentence:  Kolkatta 
-
Input sentence: Capital of haryana?
Decoded sentence:  chandigarh 
-
Input sentence: Capital of Gujarat?
Decoded sentence:  Gandinagar 
-
Input sentence: Capital of punjab?
Decoded sentence:  chandigarh 
-
Input sentence: capital of orissa?
Decoded sentence:  buvaneshwar 
-
Input sentence: capital of jammu?
Decoded sentence:  jammu 
-
Input sentence: Capital of rajastan?
Decoded sentence:  jaipur 
-
Input sentence: capital of nagaland?
Decoded sentence:  nagaland 
-
Input sentence: capital of assam?
Decoded sentence:  dispur 
-
Input sentence: capital of france?
Decoded sentence:  paris 
-
Input sentence: capital of finland?
Decoded sentence:  helsinki